In [1]:
import os

In [2]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '改价表' in file:
        price = './input/' + file
    elif 'position' in file:
        position = './input/' + file
    elif 'colorless' in file:
        this = './input/' + file

print('price:         ' + price)
print('position:      ' + position)
print('this:          ' + this)

price:         ./input/改价表5.3.xlsx
position:      ./input/1.position.xlsx
this:          ./input/am-colorless.xlsx


In [3]:
import pandas as pd

In [4]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         dtype=str).fillna('')[['SKU', '数量']]

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['数量'] = df_price['数量'].astype(int)
df_price = df_price.groupby(df_price['SKU'], as_index=False).sum()

df_price

,SKU,数量
0,USABS00417-C,2
1,USABS05515,2
2,USABS10770-C,1
3,USABS11470-C,1
4,USABS11490-C,1
...,...,...
2349,USWSP4TG01B0-4-YC,1
2350,USWSP53902B1-4-YC,1
2351,USWSP54712B1-2-YC,1
2352,USWSP54716B1-2-YC,2


In [5]:
df_position = pd.read_excel(position,
                            header=0,
                            dtype=str).fillna('')

df_position

,主料,SKU,row
0,USAC64980T,USAC64980T,1
1,USACC02805S-C,USACC02805S-C,2
2,USACC29325-C,USACC29325-C,3
3,USACH11594J-C,USACH11594J-C,4
4,USOC51647,USOC51647,5
...,...,...,...
21122,USGS62390GUB,USGS62390GUB-C,8239
21123,USGS6310TNZB-C,USGS6310NZ,8288
21124,USGS64520TNZ-C,USGS64520,8370
21125,USGS65180TB-C,USGS65180T,8407


In [6]:
print(df_price.columns)
print(df_position.columns)

df_price = pd.merge(left=df_price,
                    right=df_position,
                    how='left',
                    left_on='SKU',
                    right_on='SKU').fillna('')

df_price

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,SKU,数量,主料,row
0,USABS00417-C,2,,
1,USABS05515,2,,
2,USABS10770-C,1,,
3,USABS11470-C,1,,
4,USABS11490-C,1,,
...,...,...,...,...
2349,USWSP4TG01B0-4-YC,1,,
2350,USWSP53902B1-4-YC,1,,
2351,USWSP54712B1-2-YC,1,,
2352,USWSP54716B1-2-YC,2,,


In [7]:
# 找出有position关系的数据
df_temp = df_price[df_price['row'] != '']
df_temp = df_temp[['row', '数量']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,数量
0,1002,2
1,1007,1
2,1014,3
3,102,1
4,1100,1
...,...,...
515,980,1
516,981,1
517,983,1
518,998,1


In [8]:
print(df_price.columns)
print(df_temp.columns)

# 更新有position关系的数据
df_price = pd.merge(left=df_price,
                    right=df_temp,
                    how='left',
                    left_on='row',
                    right_on='row',
                    suffixes=('', '_temp')).fillna('')

df_price['数量'] = [amount if amount_temp == '' else amount_temp for amount, amount_temp in zip(df_price['数量'].tolist(), df_price['数量_temp'].tolist())]

df_price = df_price[['SKU', '数量']]

df_price

Index(['SKU', '数量', '主料', 'row'], dtype='object')
Index(['row', '数量'], dtype='object')


,SKU,数量
0,USABS00417-C,2.0
1,USABS05515,2.0
2,USABS10770-C,1.0
3,USABS11470-C,1.0
4,USABS11490-C,1.0
...,...,...
2349,USWSP4TG01B0-4-YC,1.0
2350,USWSP53902B1-4-YC,1.0
2351,USWSP54712B1-2-YC,1.0
2352,USWSP54716B1-2-YC,2.0


In [9]:
df_this = pd.read_excel(this,
                        header=1,
                        usecols='D',
                        dtype=str).fillna('')

df_this

,SKU
0,USACB00278
1,USACC68322-C
2,USACC78321-C
3,USACC1G010-C
4,USACC98442-C
...,...
13746,USGS49520LR
13747,USGS49650
13748,USGS64080
13749,USGSM1048


In [10]:
print(df_this.columns)
print(df_price.columns)

df_this = pd.merge(left=df_this,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)

df_this

Index(['SKU'], dtype='object')
Index(['SKU', '数量'], dtype='object')


,SKU,数量
0,USACB00278,0.0
1,USACC68322-C,0.0
2,USACC78321-C,0.0
3,USACC1G010-C,0.0
4,USACC98442-C,0.0
...,...,...
13746,USGS49520LR,0.0
13747,USGS49650,0.0
13748,USGS64080,0.0
13749,USGSM1048,0.0


In [11]:
from datetime import datetime

In [12]:
df_this.to_excel('./补充-' + datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

print('Done ~')

Done ~
